# Importação das bibliotecas

In [1]:
import pandas as pd

# Carregamento de dados

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Dados de Mapbiomas

COBERTURA_COL7

In [3]:
pd.read_excel('https://storage.googleapis.com/mapbiomas-public/brasil/downloads/1-tabela-geral-col7-mapbiomas-biomas-municipio-final.xlsx', #Extraindo dados da fonte (excel)
              sheet_name='COBERTURA_COL7').to_csv("/content/drive/MyDrive/Dados/DW_Cobertura-da-terra/Cobertura.csv",
                                                  index=False) # Salvando dados em .CSV

## Dados de microregiões (IBGE)

Microregiões

In [4]:
pd.read_excel('https://geoftp.ibge.gov.br/organizacao_do_territorio/divisao_regional/divisao_regional_do_brasil/divisao_regional_do_brasil_em_regioes_geograficas_2017/tabelas/regioes_geograficas_composicao_por_municipios_2017_20180911.xlsx', #Extraindo dados da fonte (excel)
              sheet_name='Planilha1').to_csv('/content/drive/MyDrive/Dados/dados-de-apoio/Microregioes.csv', 
                                             index=False) # Salvando dados em .CSV

# Visualizando dados em CSV

## Dados de uso da terra

### Limpeza de dados

In [5]:
dados_brutos = pd.read_csv('/content/drive/MyDrive/Dados/DW_Cobertura-da-terra/Cobertura.csv') # Carregando dados de uso da terra

# ------------------- Limpeza para retirada de espaços --------------------
df_obj = dados_brutos.select_dtypes(['object']) 
dados_brutos[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())

In [6]:
dados_brutos.head()

,feature_id,municipality,UF,biome,class_id,level_0,level_1,level_2,level_3,level_4,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,17694,Alta Floresta D'Oeste,RO,AMAZÔNIA,3,Natural,1. Forest,Forest Formation,Forest Formation,Forest Formation,...,384502.647857,385731.664005,386329.502617,384468.101447,382236.220577,381012.501440,378469.144588,376147.297128,373638.482365,367286.801569
1,17694,Alta Floresta D'Oeste,RO,AMAZÔNIA,4,Natural,1. Forest,Savanna Formation,Savanna Formation,Savanna Formation,...,23133.612748,22926.775824,22909.689889,22951.868474,22930.509200,23064.354384,23156.676244,23233.372206,22489.009435,7320.889806
2,17694,Alta Floresta D'Oeste,RO,AMAZÔNIA,11,Natural,2. Non Forest Natural Formation,Wetland,Wetland,Wetland,...,3457.880837,3743.130015,3219.649136,2200.362488,2225.866810,1942.799180,1826.344285,3052.745465,1449.423688,557.166324
3,17694,Alta Floresta D'Oeste,RO,AMAZÔNIA,12,Natural,2. Non Forest Natural Formation,Grassland,Grassland,Grassland,...,86563.678701,87516.487712,87633.121536,89587.127531,89573.015839,90146.941530,89866.198953,88468.452332,92539.203490,87873.294411
4,17694,Alta Floresta D'Oeste,RO,AMAZÔNIA,15,Anthropic,3. Farming,Pasture,Pasture,Pasture,...,201974.097430,200969.601745,200364.127425,201796.551448,203818.420842,204249.973320,206720.865824,208872.295572,209896.319951,237003.933246


In [7]:
dados_brutos.level_4.unique()

array(['Forest Formation', 'Savanna Formation', 'Wetland', 'Grassland',
       'Pasture', 'Soy Beans', 'Other Temporary Crops',
       'Urban Infrastructure', 'River, Lake and Ocean', 'Non Observed',
       'Mining', 'Forest Plantation', 'Cotton', 'Land Use Mosaic',
       'Other Non Vegetated Area', 'Sugar Cane', 'Salt flat',
       'Other Perennial Crops', 'Magrove', 'Beach and Dune',
       'Rocky outcrop', 'Rice', 'Aquaculture', 'Coffe', 'Citrus',
       'Wooded Restinga', 'Restinga Herbácea/Arbustiva',
       'Other Non Forest Natural Formation'], dtype=object)

In [14]:
dict_classes = dict(zip(dados_brutos.level_4.unique(), dados_brutos.class_id.unique()))
dict_classes

{'Forest Formation': 3,
 'Savanna Formation': 4,
 'Wetland': 11,
 'Grassland': 12,
 'Pasture': 15,
 'Soy Beans': 39,
 'Other Temporary Crops': 41,
 'Urban Infrastructure': 24,
 'River, Lake and Ocean': 33,
 'Non Observed': 0,
 'Mining': 30,
 'Forest Plantation': 9,
 'Cotton': 62,
 'Land Use Mosaic': 21,
 'Other Non Vegetated Area': 25,
 'Sugar Cane': 20,
 'Salt flat': 32,
 'Other Perennial Crops': 48,
 'Magrove': 5,
 'Beach and Dune': 23,
 'Rocky outcrop': 29,
 'Rice': 40,
 'Aquaculture': 31,
 'Coffe': 46,
 'Citrus': 47,
 'Wooded Restinga': 49,
 'Restinga Herbácea/Arbustiva': 50,
 'Other Non Forest Natural Formation': 13}

### Seleção de categorias

In [17]:
classes_de_interesse = ['Forest Formation', 'Pasture', 
                        'Soy Beans', 'Other Temporary Crops',
                        'Forest Plantation', 'Other Perennial Crops']

lista_filtrados = []

for i in classes_de_interesse:
    
    lista_filtrados.append(dados_brutos.loc[(dados_brutos.class_id == dict_classes[i]) & (dados_brutos.UF == 'PA')])

dados_filtrados = pd.concat(lista_filtrados)[['municipality', 'class_id','level_4', '1985', '1986',
                                            '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995',
                                            '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004',
                                            '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
                                            '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021']].fillna(0)

dados_filtrados.head()


,municipality,class_id,level_4,1985,1986,1987,1988,1989,1990,1991,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
1406,Abaetetuba,3,Forest Formation,100546.743913,101306.542189,100384.289269,99724.925135,99667.925424,100683.032753,99838.267334,...,96645.692562,95483.644518,94251.417993,93839.921283,93176.085484,92302.799622,92729.301835,93532.710235,93580.159176,93363.897514
1419,Abel Figueiredo,3,Forest Formation,26613.814579,26579.326800,23894.794129,22440.896160,21498.310526,22717.282657,23184.053042,...,17441.244610,18017.202259,18307.422068,17737.428060,18242.843335,17986.726291,17343.938681,17761.711567,16996.719810,16190.669941
1430,Acará,3,Forest Formation,371203.416897,375702.005732,361780.976273,349991.190840,348903.064040,352838.552263,357525.820298,...,306402.870390,302571.348651,294951.735171,292390.445426,287015.506881,280509.990573,273966.581313,279916.643576,279210.868308,275427.401513
1442,Afuá,3,Forest Formation,528751.677444,534526.614411,535716.198428,533561.926656,516069.824957,531336.429793,531011.094247,...,536910.064328,537422.479373,537209.447656,536790.272654,536867.418641,537150.624894,536523.427366,537048.803191,537029.757964,538597.224720
1451,Água Azul do Norte,3,Forest Formation,563593.085978,563893.099210,551319.439805,538699.826324,525662.639282,514628.868402,505840.550850,...,285093.892212,288043.688949,285488.630513,283014.812285,278862.783526,273875.114344,276052.724315,278033.178828,281711.833873,288030.016399


## Dados de micro/macro-regiões

In [ ]:
micro_reg = pd.read_csv('/content/drive/MyDrive/Dados/dados-de-apoio/Microregioes.csv') # Carregando dados de microregiões
micro_reg.rename(columns={'nome_mun':'municipality', 'CD_GEOCODI':'cod_municipality'}, inplace=True) # Renomeando dados para mesclagem
micro_reg.head()

,municipality,cod_municipality,cod_rgi,nome_rgi,cod_rgint,nome_rgint
0,Itapuã do Oeste,1101104,110001,Porto Velho,1101,Porto Velho
1,Candeias do Jamari,1100809,110001,Porto Velho,1101,Porto Velho
2,Nova Mamoré,1100338,110001,Porto Velho,1101,Porto Velho
3,Porto Velho,1100205,110001,Porto Velho,1101,Porto Velho
4,Guajará-Mirim,1100106,110001,Porto Velho,1101,Porto Velho


# Mesclagem de dados (Microregiões)

In [ ]:
dados_merge = dados_filtrados.merge(micro_reg, how = 'inner', on='municipality') # Mesclagem de dados
dados_merge.head()

,municipality,class_id,level_4,1985,1986,1987,1988,1989,1990,1991,...,2017,2018,2019,2020,2021,cod_municipality,cod_rgi,nome_rgi,cod_rgint,nome_rgint
0,Abaetetuba,3,Forest Formation,100546.743913,101306.542189,100384.289269,99724.925135,99667.925424,100683.032753,99838.267334,...,92302.799622,92729.301835,93532.710235,93580.159176,93363.897514,1500107,150003,Abaetetuba,1501,Belém
1,Abaetetuba,15,Pasture,5044.903800,4139.367871,5002.560373,5623.327225,5502.784853,4733.681792,6110.376955,...,14916.967394,14151.090292,13445.343754,13185.143045,13071.973225,1500107,150003,Abaetetuba,1501,Belém
2,Abaetetuba,39,Soy Beans,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,2.680636,1500107,150003,Abaetetuba,1501,Belém
3,Abaetetuba,41,Other Temporary Crops,0.000000,0.089356,0.089356,0.089356,1.161644,1.161644,1.072288,...,35.205075,28.235472,21.087233,22.516923,55.489021,1500107,150003,Abaetetuba,1501,Belém
4,Abaetetuba,48,Other Perennial Crops,638.716826,638.270027,687.682811,806.790139,1006.935503,842.961131,792.741740,...,2353.357306,2329.767996,2327.712880,2326.283216,2325.121680,1500107,150003,Abaetetuba,1501,Belém


Cobertura_e_microreg

In [ ]:
dados_merge.columns

Index(['municipality', 'class_id', 'level_4', '1985', '1986', '1987', '1988',
       '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997',
       '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006',
       '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015',
       '2016', '2017', '2018', '2019', '2020', '2021', 'cod_municipality',
       'cod_rgi', 'nome_rgi', 'cod_rgint', 'nome_rgint'],
      dtype='object')

In [ ]:
dados_merge = dados_merge[['municipality', 'cod_municipality','cod_rgint','nome_rgint', 'cod_rgi', 'nome_rgi', 'class_id', 'level_4', 
                           '1985', '1986', '1987', '1988',
                          '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997',
                          '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006',
                          '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015',
                          '2016', '2017', '2018', '2019', '2020', '2021']]

In [ ]:
dados_merge.head()

,municipality,cod_municipality,cod_rgint,nome_rgint,cod_rgi,nome_rgi,class_id,level_4,1985,1986,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Abaetetuba,1500107,1501,Belém,150003,Abaetetuba,3,Forest Formation,100546.743913,101306.542189,...,96645.692562,95483.644518,94251.417993,93839.921283,93176.085484,92302.799622,92729.301835,93532.710235,93580.159176,93363.897514
1,Abaetetuba,1500107,1501,Belém,150003,Abaetetuba,15,Pasture,5044.903800,4139.367871,...,11054.932377,11603.950761,13055.996662,13341.689949,13942.802844,14916.967394,14151.090292,13445.343754,13185.143045,13071.973225
2,Abaetetuba,1500107,1501,Belém,150003,Abaetetuba,39,Soy Beans,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.680636
3,Abaetetuba,1500107,1501,Belém,150003,Abaetetuba,41,Other Temporary Crops,0.000000,0.089356,...,31.184249,33.239327,27.073903,32.435191,31.184104,35.205075,28.235472,21.087233,22.516923,55.489021
4,Abaetetuba,1500107,1501,Belém,150003,Abaetetuba,48,Other Perennial Crops,638.716826,638.270027,...,1099.478910,2479.081339,2522.146984,2543.948126,2592.109960,2353.357306,2329.767996,2327.712880,2326.283216,2325.121680


In [ ]:
dados_merge.rename(columns = {'municipality':'municipio', 'class_id':'id_classe',
                              'nome_rgint':'regiao_intermediaria', 'cod_rgint':'cod_regiao_intermediaria',
                              'nome_rgi':'regiao_imediata',
                              'cod_rgi':'cod_regiao_imediata',
                              'level_4':'nome_classe'}, inplace=True)

In [ ]:
dados_merge.head(15)

,municipio,cod_municipality,cod_regiao_intermediaria,regiao_intermediaria,cod_regiao_imediata,regiao_imediata,id_classe,nome_classe,1985,1986,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Abaetetuba,1500107,1501,Belém,150003,Abaetetuba,3,Forest Formation,100546.743913,101306.542189,...,96645.692562,95483.644518,94251.417993,93839.921283,93176.085484,92302.799622,92729.301835,93532.710235,93580.159176,93363.897514
1,Abaetetuba,1500107,1501,Belém,150003,Abaetetuba,15,Pasture,5044.903800,4139.367871,...,11054.932377,11603.950761,13055.996662,13341.689949,13942.802844,14916.967394,14151.090292,13445.343754,13185.143045,13071.973225
2,Abaetetuba,1500107,1501,Belém,150003,Abaetetuba,39,Soy Beans,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.680636
3,Abaetetuba,1500107,1501,Belém,150003,Abaetetuba,41,Other Temporary Crops,0.000000,0.089356,...,31.184249,33.239327,27.073903,32.435191,31.184104,35.205075,28.235472,21.087233,22.516923,55.489021
4,Abaetetuba,1500107,1501,Belém,150003,Abaetetuba,48,Other Perennial Crops,638.716826,638.270027,...,1099.478910,2479.081339,2522.146984,2543.948126,2592.109960,2353.357306,2329.767996,2327.712880,2326.283216,2325.121680
5,Abaetetuba,1500107,1501,Belém,150003,Abaetetuba,9,Forest Plantation,0.714807,0.714807,...,253.665671,284.760714,283.956512,284.403231,284.403204,285.207362,285.296693,285.386031,284.849893,284.135101
6,Abel Figueiredo,1500131,1503,Marabá,150009,Marabá,3,Forest Formation,26613.814579,26579.326800,...,17441.244610,18017.202259,18307.422068,17737.428060,18242.843335,17986.726291,17343.938681,17761.711567,16996.719810,16190.669941
7,Abel Figueiredo,1500131,1503,Marabá,150009,Marabá,15,Pasture,34186.867983,34230.887086,...,42860.849981,42324.961991,42034.204538,42519.756945,41739.370686,41605.964035,41686.193531,41181.467913,41915.195335,42946.616598
8,Abel Figueiredo,1500131,1503,Marabá,150009,Marabá,39,Soy Beans,0.000000,0.000000,...,0.000000,0.000000,27.704498,28.149907,62.090445,136.296160,252.994401,765.677411,803.538297,751.322869
9,Abel Figueiredo,1500131,1503,Marabá,150009,Marabá,41,Other Temporary Crops,0.000000,0.000000,...,116.584284,42.574666,24.493611,110.806420,301.153651,550.563389,977.365416,515.828505,490.439763,359.770903


cobertura_formatados

In [ ]:
dados_merge.to_csv('/content/drive/MyDrive/Dados/DW_Cobertura-da-terra/cobertura_formatados.csv', 
                   index = False) # Salvando dados em CSV

# Tabelas

## 1612 - Lavouras temporárias

### Informação sobre dados 

**/P/**

Ano(48):  1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021
<br><br><br><br>
**/C81/  Produto das lavouras temporárias(34):** <br><br>
2688  Abacaxi* <br>
40471  Alfafa fenada [1974 a 1987] <br>
2689  Algodão herbáceo (em caroço) <br>
2690  Alho <br>
2691  Amendoim (em casca) <br>
2692  Arroz (em casca) <br>
2693  Aveia (em grão) <br>
2694  Batata-doce <br>
2695  Batata-inglesa <br>
~2696  Cana-de-açúcar~ <br>
~40470  Cana para forragem~ [1974 a 1987] <br>
2697  Cebola <br>
2698  Centeio (em grão) <br>
2699  Cevada (em grão) <br>
2700  Ervilha (em grão) [1988 a 2021] <br>
2701  Fava (em grão) <br>
2702  Feijão (em grão) <br>
2703  Fumo (em folha) <br>
109179  Girassol (em grão) [2005 a 2021] <br>
2704  Juta (fibra) <br>
2705  Linho (semente) <br>
2706  Malva (fibra) <br>
2707  Mamona (baga) <br>
2708  Mandioca <br>
2709  Melancia <br>
2710  Melão <br>
2711  Milho (em grão) <br>
2712  Rami (fibra) <br>
2713  Soja (em grão) <br>
2714  Sorgo (em grão) <br>
2715  Tomate <br>
2716  Trigo (em grão) <br>
109180  Triticale (em grão) [2005 a 2021] <br><br>

Totalização de categorias não disponível para as variáveis 'Quantidade produzida', 'Rendimento médio da produção'.

Soma de categorias não disponível para a variável 'Rendimento médio da produção'.
Níveis Territoriais:

/N1/ Brasil(1)   Listar unidades territoriais

/N2/ Grande Região(5)   Listar unidades territoriais

/N3/ Unidade da Federação(27)   Listar unidades territoriais

/N8/ Mesorregião Geográfica [1990 a 2021](137)   Listar unidades territoriais

/N9/ Microrregião Geográfica [1990 a 2021](558)   Listar unidades territoriais

*/N6/ Município(5.563)   Listar unidades territoriais*
<br><br><br><br>
Nota:


1 - Os municípios sem informação para pelo menos um produto da lavoura temporária não aparecem nas listas;

2 - A partir do ano de 2001 as quantidades produzidas dos produtos melancia e melão passam a ser expressas em toneladas. Nos anos anteriores eram expressas em mil frutos. O rendimento médio passa a ser expresso em Kg/ha. Nos anos anteriores era expresso em frutos/ha.

3 - Veja o documento AlteracoesUnidadesMedidaFrutas.pdf com as alterações de unidades de medida das frutíferas ocorridas em 2001 e a tabela de conversão fruto x quilograma.

4 - Os produtos girassol e triticale só apresentam informação a partir de 2005.

5 - A quantidade produzida de abacaxi é expressa em mil frutos e o rendimento médio em frutos/ha.

6 - Valores para a categoria Total indisponíveis para as variáveis Quantidade produzida e Rendimento médio, pois as unidades de medida diferem para determinados produtos.

7 - Subentende a possibilidade de cultivos sucessivos ou simultâneos (simples, associados e/ou intercalados) no mesmo ano e no mesmo local, podendo, por isto, a área informada da cultura exceder a área geográfica do município.

8 - As culturas de abacaxi, cana-de-açúcar, mamona e mandioca são consideradas culturas temporárias de longa duração. Elas costumam ter ciclo vegetativo que ultrapassa 12 meses e, por isso, as informações são computadas nas colheitas realizadas dentro de cada ano civil (12 meses).
Nestas culturas a área plantada refere-se a área destinada à colheita no ano.

9 - A diferença entre a área plantada e a área colhida na lavoura temporária é considerada como área perdida.

10 - A variável Área plantada só passou a ser informada a partir de 1988.

11 - Valor da produção: Variável derivada calculada pela média ponderada das informações de quantidade e preço médio corrente pago ao produtor, de acordo com os períodos de colheita e comercialização de cada produto. As despesas de frete, taxas e impostos não são incluídas no preço.

12 - Os dados do último ano divulgado são RESULTADOS PRELIMINARES e podem sofrer alterações até a próxima divulgação.



### Extração dos dados

In [ ]:

a = []
lavouras_temporarias = [2688, 2691, 2692,2694,2696,2697,2702,2703,2706,2706,2709,2711,2713,2714,2715]

for j in lavouras_temporarias:
  link = f"https://apisidra.ibge.gov.br/values/t/1612/n6/in%20n3%2015/p/all/v/214,109,216/c81/{j}/h/n"
  try:
    print(f'tentando: {j}')
    df = pd.read_json(link)
    print(f'concluido: {j}')
    a.append(df)
  except:
    
    print(f'falha: {j}')

table_1612 = pd.concat(a)
table_1612

tentando: 2688
concluido: 2688
tentando: 2691
concluido: 2691
tentando: 2692
concluido: 2692
tentando: 2694
concluido: 2694
tentando: 2696
concluido: 2696
tentando: 2697
concluido: 2697
tentando: 2702
concluido: 2702
tentando: 2703
concluido: 2703
tentando: 2706
concluido: 2706
tentando: 2706
concluido: 2706
tentando: 2709
concluido: 2709
tentando: 2711
concluido: 2711
tentando: 2713
concluido: 2713
tentando: 2714
concluido: 2714
tentando: 2715
concluido: 2715


,NC,NN,MC,MN,V,D1C,D1N,D2C,D2N,D3C,D3N,D4C,D4N
0,6,Município,1017,Toneladas,-,1500107,Abaetetuba - PA,1974,1974,214,Quantidade produzida,2688,Abacaxi*
1,6,Município,,,...,1500107,Abaetetuba - PA,1974,1974,109,Área plantada,2688,Abacaxi*
2,6,Município,1006,Hectares,-,1500107,Abaetetuba - PA,1974,1974,216,Área colhida,2688,Abacaxi*
3,6,Município,1017,Toneladas,-,1500107,Abaetetuba - PA,1975,1975,214,Quantidade produzida,2688,Abacaxi*
4,6,Município,,,...,1500107,Abaetetuba - PA,1975,1975,109,Área plantada,2688,Abacaxi*
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20587,6,Município,1006,Hectares,-,1508407,Xinguara - PA,2020,2020,109,Área plantada,2715,Tomate
20588,6,Município,1006,Hectares,-,1508407,Xinguara - PA,2020,2020,216,Área colhida,2715,Tomate
20589,6,Município,1017,Toneladas,-,1508407,Xinguara - PA,2021,2021,214,Quantidade produzida,2715,Tomate
20590,6,Município,1006,Hectares,-,1508407,Xinguara - PA,2021,2021,109,Área plantada,2715,Tomate


In [ ]:
table_1612 = pd.concat(a)
table_1612

,NC,NN,MC,MN,V,D1C,D1N,D2C,D2N,D3C,D3N,D4C,D4N
0,6,Município,1017,Toneladas,-,1500107,Abaetetuba - PA,1974,1974,214,Quantidade produzida,2688,Abacaxi*
1,6,Município,,,...,1500107,Abaetetuba - PA,1974,1974,109,Área plantada,2688,Abacaxi*
2,6,Município,1006,Hectares,-,1500107,Abaetetuba - PA,1974,1974,216,Área colhida,2688,Abacaxi*
3,6,Município,1017,Toneladas,-,1500107,Abaetetuba - PA,1975,1975,214,Quantidade produzida,2688,Abacaxi*
4,6,Município,,,...,1500107,Abaetetuba - PA,1975,1975,109,Área plantada,2688,Abacaxi*
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20587,6,Município,1006,Hectares,-,1508407,Xinguara - PA,2020,2020,109,Área plantada,2715,Tomate
20588,6,Município,1006,Hectares,-,1508407,Xinguara - PA,2020,2020,216,Área colhida,2715,Tomate
20589,6,Município,1017,Toneladas,-,1508407,Xinguara - PA,2021,2021,214,Quantidade produzida,2715,Tomate
20590,6,Município,1006,Hectares,-,1508407,Xinguara - PA,2021,2021,109,Área plantada,2715,Tomate


In [ ]:
table_1612["Tipo de produção"] = "Produto das lavouras temporárias"

In [ ]:
table_1612_copy = table_1612.copy()

table_1612_copy.columns = ['cod_mun', 'mun', 'cod_metrica', 'metrica', 'valor', 'cod_cidade', 'cidade', 'cod_ano', 'ano', 'cod_variavel', 'variavel', 'cod_produto', 'produto', 'tipo_producao']

In [ ]:
table_1612 = table_1612_copy

table_1612.to_csv("/content/drive/MyDrive/Dados/Dados_SIDRA/table_1612-lavouras_temporarias.csv", index= False)

## 1613 - Lavouras permanentes

### Informação sobre dados

/T/  Tabela:  	1613 - 	Área destinada à colheita, área colhida, quantidade produzida, rendimento médio e valor da produção das lavouras permanentes
Pesquisa:  Produção Agrícola Municipal<br>
<br><br>
Assunto:  Lavouras permanentes
<br><br>
Última atualização:  2022-09-15 09:46:58
<br><br>
/P/  Ano(48):  1974 - 2021
<br><br>
/V/  Variável(8):
<br><br>
2313  Área destinada à colheita (Hectares) [1988 a 2021] - casas decimais: padrão = 0, máximo = 0<br>
1002313  Área destinada à colheita - percentual do total geral (%) [1988 a 2021] - casas decimais: padrão = 2, máximo = 5<br>
216  Área colhida (Hectares) - casas decimais: padrão = 0, máximo = 0 <br>
1000216  Área colhida - percentual do total geral (%) - casas decimais: padrão = 2, máximo = 5<br>
214  Quantidade produzida (Toneladas) - casas decimais: padrão = 0, máximo = 0<br>
112  Rendimento médio da produção (Quilogramas por Hectare) - casas decimais: padrão = 0, máximo = 0<br>
215  Valor da produção (Mil Cruzeiros [1974 a 1985, 1990 a 1992], Mil Cruzados [1986 a 1988], Mil Cruzados Novos [1989], Mil Cruzeiros Reais [1993], Mil Reais [1994 a 2021]) - casas decimais: padrão = 0, máximo = 0<br>
1000215  Valor da produção - percentual do total geral (%) - casas decimais: padrão = 2, máximo = 5<br>
<br><br>
/C82/  Produto das lavouras permanentes(39):<br><br>
0  Total <br>
2717  Abacate<br>
2718  Algodão arbóreo (em caroço)<br>
45981  Açaí [2015 a 2021]<br>
2719  Azeitona<br>
2720  Banana (cacho)<br>
2721  Borracha (látex coagulado) [1981 a 2021]<br>
40472  Borracha (látex líquido) [1981 a 1987]<br>
2722  Cacau (em amêndoa)<br>
2723  Café (em grão) Total<br>
31619  Café (em grão) Arábica [2012 a 2021]<br>
31620  Café (em grão) Canephora [2012 a 2021]<br>
40473  Caju [1974 a 1987]<br>
2724  Caqui<br>
2725  Castanha de caju [1988 a 2021]<br>
2726  Chá-da-índia (folha verde)<br>
2727  Coco-da-baía*<br>
2728  Dendê (cacho de coco) [1988 a 2021]<br>
2729  Erva-mate (folha verde) [1981 a 2021]<br>
2730  Figo<br>
2731  Goiaba [1988 a 2021]<br>
2732  Guaraná (semente) [1981 a 2021]<br>
2733  Laranja<br>
2734  Limão<br>
2735  Maçã<br>
2736  Mamão<br>
2737  Manga<br>
2738  Maracujá [1988 a 2021]<br>
2739  Marmelo<br>
2740  Noz (fruto seco)<br>
90001  Palmito [1981 a 2021]<br>
2741  Pera<br>
2742  Pêssego<br>
2743  Pimenta-do-reino<br>
2744  Sisal ou agave (fibra)<br>
2745  Tangerina<br>
2746  Tungue (fruto seco)<br>
2747  Urucum (semente) [1988 a 2021]<br>
2748  Uva<br><br>

Totalização de categorias não disponível para as variáveis 'Quantidade produzida', 'Rendimento médio da produção'.
<br>
Soma de categorias não disponível para a variável 'Rendimento médio da produção'.
Níveis Territoriais:
<br>
/N1/ Brasil(1)   Listar unidades territoriais<br>
/N2/ Grande Região(5)   Listar unidades territoriais<br>
/N3/ Unidade da Federação(27)   Listar unidades territoriais<br>
/N8/ Mesorregião Geográfica [1990 a 2021](137)   Listar unidades territoriais<br>
/N9/ Microrregião Geográfica [1990 a 2021](557)   Listar unidades territoriais<br>
/N6/ Município(5.533)   Listar unidades territoriais<br>
<br>
Nota:
<br><br>
1 - Os municípios sem informação para pelo menos um produto da lavoura permanente não aparecem nas listas.
<br><br>
2 - A partir do ano de 2001 as quantidades produzidas dos produtos abacate, banana, caqui, figo, goiaba, laranja, limão, maçã, mamão, manga, maracujá, marmelo, pera, pêssego e tangerina passam a ser expressas em toneladas. Nos anos anteriores eram expressas em mil frutos, com exceção da banana, que era expressa em mil cachos. O rendimento médio passa a ser expresso em Kg/ha. Nos anos anteriores era expresso em frutos/ha, com exceção da banana, que era expressa em cachos/ha.
<br><br>
3 - Veja em o documento AlteracoesUnidadesMedidaFrutas.pdf com as alterações de unidades de medida das frutíferas ocorridas em 2001 e a tabela de conversão fruto x quilograma.
<br><br>
4 - Até 2001, café (em coco), a partir de 2002, café (beneficiado ou em grão).
<br><br>
5 - A quantidade produzida de coco-da-baía é expressa em mil frutos e o rendimento médio em frutos/ha.
<br><br>
6 - Valores para a categoria Total indisponíveis para a variável Quantidade produzida e Rendimento médio, pois as unidades de medida diferem para determinados produtos.
<br><br>
7 - Subentende a possibilidade de cultivos sucessivos ou simultâneos (simples, associados e/ou intercalados) no mesmo ano e no mesmo local, podendo, por isto, a área informada da cultura exceder a área geográfica do município.
<br><br>
8 - A diferença entre a área destinada à colheita e a área colhida na lavoura permanente é considerada como área perdida.
<br><br>
9 - A variável Área destinada à colheita só passou a ser informada a partir de 1988.
<br><br>
10 - Valor da produção: Variável derivada calculada pela média ponderada das informações de quantidade e preço médio corrente pago ao produtor, de acordo com os períodos de colheita e comercialização de cada produto. As despesas de frete, taxas e impostos não são incluídas no preço.
<br><br>
11 - Os dados do último ano divulgado são RESULTADOS PRELIMINARES e podem sofrer alterações até a próxima divulgação.
<br><br>
 

### Extração dos dados

In [ ]:
a = []
produto_das_lavouras_permanentes = [45981,2720,2721,2722,2723,31619,2725,2727,2731,2733,2734,2736,2738,90001,2743,2745,2747]


for j in produto_das_lavouras_permanentes:
  link = f"https://apisidra.ibge.gov.br/values/t/1613/n6/in%20n3%2015/p/all/v/216,2313,214/c82/{j}/h/n"
  try:
    print(f'tentando: {j}')
    df = pd.read_json(link)
    print(f'concluido: {j}')
    a.append(df)
  except:
    
    print(f'falha: {j}')

table_1613 = pd.concat(a)
table_1613

tentando: 45981
concluido: 45981
tentando: 2720
concluido: 2720
tentando: 2721
concluido: 2721
tentando: 2722
concluido: 2722
tentando: 2723
concluido: 2723
tentando: 31619
concluido: 31619
tentando: 2725
concluido: 2725
tentando: 2727
concluido: 2727
tentando: 2731
concluido: 2731
tentando: 2733
concluido: 2733
tentando: 2734
concluido: 2734
tentando: 2736
concluido: 2736
tentando: 2738
concluido: 2738
tentando: 90001
concluido: 90001
tentando: 2743
concluido: 2743
tentando: 2745
concluido: 2745
tentando: 2747
concluido: 2747


,NC,NN,MC,MN,V,D1C,D1N,D2C,D2N,D3C,D3N,D4C,D4N
0,6,Município,1006,Hectares,...,1500107,Abaetetuba - PA,1974,1974,216,Área colhida,45981,Açaí
1,6,Município,,,...,1500107,Abaetetuba - PA,1974,1974,2313,Área destinada à colheita,45981,Açaí
2,6,Município,1017,Toneladas,...,1500107,Abaetetuba - PA,1974,1974,214,Quantidade produzida,45981,Açaí
3,6,Município,1006,Hectares,...,1500107,Abaetetuba - PA,1975,1975,216,Área colhida,45981,Açaí
4,6,Município,,,...,1500107,Abaetetuba - PA,1975,1975,2313,Área destinada à colheita,45981,Açaí
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20587,6,Município,1006,Hectares,-,1508407,Xinguara - PA,2020,2020,2313,Área destinada à colheita,2747,Urucum (semente)
20588,6,Município,1017,Toneladas,-,1508407,Xinguara - PA,2020,2020,214,Quantidade produzida,2747,Urucum (semente)
20589,6,Município,1006,Hectares,-,1508407,Xinguara - PA,2021,2021,216,Área colhida,2747,Urucum (semente)
20590,6,Município,1006,Hectares,-,1508407,Xinguara - PA,2021,2021,2313,Área destinada à colheita,2747,Urucum (semente)


In [ ]:
table_1613['tipo_producao'] = 'Produto das lavouras permanentes'

table_1613_column_names = ['cod_mun', 'mun', 'cod_metrica', 'metrica', 
                           'valor', 'cod_cidade', 'cidade', 'cod_ano', 
                           'ano', 'cod_variavel', 'variavel', 'cod_produto', 
                           'produto', 'tipo_producao']

table_1613.columns = table_1613_column_names

table_1613.to_csv("/content/drive/MyDrive/Dados/Dados_SIDRA/table_1613-lavouras_permanentes.csv", index= False)

## 291 - Sivilcultura

### Informação sobre dados

/T/  Tabela:  	291 - 	Quantidade produzida e valor da produção na silvicultura, por tipo de produto da silvicultura
Pesquisa:  Produção da Extração Vegetal e da Silvicultura <br><br>

Assunto:  Silvicultura<br><br>

Última atualização:  2022-09-29 09:59:59<br><br>

/P/  Ano(36):  1986 - 2021 <br><br>

/C194/  Tipo de produto da silvicultura(22):<br><br>
0  Total<br>
3455  1.1 - Carvão vegetal (Toneladas)<br>
33247  1.1.1 - Carvão vegetal de eucalipto (Toneladas) [2013 a 2021]<br>
33248  1.1.2 - Carvão vegetal de pinus (Toneladas) [2013 a 2021]<br>
33249  1.1.3 - Carvão vegetal de outras espécies (Toneladas) [2013 a 2021]<br>
3456  1.2 - Lenha (Metros cúbicos)<br>
33250  1.2.1 - Lenha de eucalipto (Metros cúbicos) [2013 a 2021]<br>
33251  1.2.2 - Lenha de pinus (Metros cúbicos) [2013 a 2021]<br>
33252  1.2.3 - Lenha de outras espécies (Metros cúbicos) [2013 a 2021]<br>
3457  1.3 - Madeira em tora (Metros cúbicos)<br>
3458  1.3.1 - Madeira em tora para papel e celulose (Metros cúbicos)<br>
33253  1.3.1.1 - Madeira em tora de eucalipto para papel e celulose (Metros<br> cúbicos) [2013 a 2021]<br>
33254  1.3.1.2 - Madeira em tora de pinus para papel e celulose (Metros cúbicos) [2013 a 2021]<br>
33255  1.3.1.3 - Madeira em tora de outras espécies para papel e celulose (Metros cúbicos) [2013 a 2021]<br>
3459  1.3.2 - Madeira em tora para outras finalidades (Metros cúbicos)<br>
33256  1.3.2.1 - Madeira em tora de eucalipto para outras finalidades (Metros cúbicos) [2013 a 2021]<br>
33257  1.3.2.2 - Madeira em tora de pinus para outras finalidades (Metros cúbicos) [2013 a 2021]<br>
33258  1.3.2.3 - Madeira em tora de outras espécies para outras finalidades (Metros cúbicos) [2013 a 2021]<br>
3460  2 - Outros produtos (Toneladas)<br>
3461  2.1 - Acácia-negra (casca) (Toneladas)<br>
3462  2.2 - Eucalipto (folha) (Toneladas)<br>
3463  2.3 - Resina (Toneladas)<br>
<br><br>

Níveis Territoriais:
<br>
/N1/ Brasil(1)   Listar unidades territoriais<br>
/N2/ Grande Região(5)   Listar unidades territoriais<br>
/N3/ Unidade da Federação(26)   Listar unidades territoriais<br>
/N8/ Mesorregião Geográfica [1990 a 2021](126)   Listar unidades territoriais<br>
/N9/ Microrregião Geográfica [1990 a 2021](451)   Listar unidades territoriais<br>
/N6/ Município(3.631)   Listar unidades territoriais<br>

Nota:
<br><br>
1 - As Unidades da Federação, mesorregiões, microrregiões e municípios sem informação para pelo menos um produto da silvicultura em pelo menos um ano da pesquisa não aparecem nas listas.
<br><br>
2 - Valor da produção na silvicultura: Variável derivada calculada pela média ponderada das informações de quantidade e preço médio corrente pago ao produtor, de acordo com os períodos de colheita e comercialização de cada produto. As despesas de frete, taxas e impostos não são incluídas no preço.
<br><br>
3 - Os dados do último ano divulgado são RESULTADOS PRELIMINARES e podem sofrer alterações até a próxima divulgação.<br><br>

### Extração dos dados

In [ ]:
produto_das_sivilcultura =[33247,33250,33253,33256,33258]

a = []
for j in produto_das_sivilcultura:
  try:
    print(f'tentando: {j}')
    df = pd.read_json(f'https://apisidra.ibge.gov.br/values/t/291/n6/in%20n3%2015/p/all/v/142/c194/{j}/h/n')
    print(f'concluído: {j}')
    a.append(df)
  except:
    print(f'falha: {j}')
table_291 = pd.concat(a)
table_291

tentando: 33247
concluído: 33247
tentando: 33250
concluído: 33250
tentando: 33253
concluído: 33253
tentando: 33256
concluído: 33256
tentando: 33258
concluído: 33258


,NC,NN,MC,MN,V,D1C,D1N,D2C,D2N,D3C,D3N,D4C,D4N
0,6,Município,,,...,1500131,Abel Figueiredo - PA,1986,1986,142,Quantidade produzida na silvicultura,33247,1.1.1 - Carvão vegetal de eucalipto
1,6,Município,,,...,1500131,Abel Figueiredo - PA,1987,1987,142,Quantidade produzida na silvicultura,33247,1.1.1 - Carvão vegetal de eucalipto
2,6,Município,,,...,1500131,Abel Figueiredo - PA,1988,1988,142,Quantidade produzida na silvicultura,33247,1.1.1 - Carvão vegetal de eucalipto
3,6,Município,,,...,1500131,Abel Figueiredo - PA,1989,1989,142,Quantidade produzida na silvicultura,33247,1.1.1 - Carvão vegetal de eucalipto
4,6,Município,,,...,1500131,Abel Figueiredo - PA,1990,1990,142,Quantidade produzida na silvicultura,33247,1.1.1 - Carvão vegetal de eucalipto
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1003,6,Município,1016,Metros cúbicos,46600,1508126,Ulianópolis - PA,2017,2017,142,Quantidade produzida na silvicultura,33258,1.3.2.3 - Madeira em tora de outras espécies p...
1004,6,Município,1016,Metros cúbicos,46500,1508126,Ulianópolis - PA,2018,2018,142,Quantidade produzida na silvicultura,33258,1.3.2.3 - Madeira em tora de outras espécies p...
1005,6,Município,1016,Metros cúbicos,18600,1508126,Ulianópolis - PA,2019,2019,142,Quantidade produzida na silvicultura,33258,1.3.2.3 - Madeira em tora de outras espécies p...
1006,6,Município,1016,Metros cúbicos,15066,1508126,Ulianópolis - PA,2020,2020,142,Quantidade produzida na silvicultura,33258,1.3.2.3 - Madeira em tora de outras espécies p...


In [ ]:
table_291['tipo_producao'] = 'Silvicultura'

table_291_column_names = ['cod_mun', 'mun', 'cod_metrica', 'metrica', 
                           'valor', 'cod_cidade', 'cidade', 'cod_ano', 
                           'ano', 'cod_variavel', 'variavel', 'cod_produto', 
                           'produto', 'tipo_producao']

table_291.to_csv("/content/drive/MyDrive/Dados/Dados_SIDRA/table_291-sivilcultura.csv", index= False)

## 3939 - Pecuária

### Informação sobre os dados

/T/  Tabela:  	3939 - 	Efetivo dos rebanhos, por tipo de rebanho
Pesquisa:  Pesquisa da Pecuária Municipal <br><br>

Assunto:  Pecuária<br><br>

Última atualização:  2022-09-22 09:37:20<br><br>

/P/  Ano(48):  1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, <br>1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, <br>1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, <br>2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, <br>2020, 2021<br><br>


/C79/  Tipo de rebanho(10):<br><br>
2670  Bovino<br>
2675  Bubalino<br>
2672  Equino<br>
32794  Suíno - total<br>
32795  Suíno - matrizes de suínos<br>
2681  Caprino<br>
2677  Ovino<br>
32796  Galináceos - total<br>
32793  Galináceos - galinhas<br>
2680  Codornas<br>
<br><br>

### Extração dos dados

In [ ]:
produto_das_pecuaria = [2670, 2675, 2672, 
                        2681, 2677]
space = ' '
a = []
for j in produto_das_pecuaria:
  link = f"https://apisidra.ibge.gov.br/values/t/3939/n6/in%20n3%2015/p/all/v/105/c79/{j}/h/n"
  try:
    print(f'tentando: {j}')
    df = pd.read_json(link)
    print(f'concluido: {j}')
    a.append(df)
  except:
    
    print(f'falha: {j}')

table_3939 = pd.concat(a)
table_3939

tentando: 2670
concluido: 2670
tentando: 2675
concluido: 2675
tentando: 2672
concluido: 2672
tentando: 2681
concluido: 2681
tentando: 2677
concluido: 2677


,NC,NN,MC,MN,V,D1C,D1N,D2C,D2N,D3C,D3N,D4C,D4N
0,6,Município,24,Cabeças,28,1500107,Abaetetuba - PA,1974,1974,105,Efetivo dos rebanhos,2670,Bovino
1,6,Município,24,Cabeças,54,1500107,Abaetetuba - PA,1975,1975,105,Efetivo dos rebanhos,2670,Bovino
2,6,Município,24,Cabeças,47,1500107,Abaetetuba - PA,1976,1976,105,Efetivo dos rebanhos,2670,Bovino
3,6,Município,24,Cabeças,39,1500107,Abaetetuba - PA,1977,1977,105,Efetivo dos rebanhos,2670,Bovino
4,6,Município,24,Cabeças,10,1500107,Abaetetuba - PA,1978,1978,105,Efetivo dos rebanhos,2670,Bovino
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6907,6,Município,24,Cabeças,1777,1508407,Xinguara - PA,2017,2017,105,Efetivo dos rebanhos,2677,Ovino
6908,6,Município,24,Cabeças,2075,1508407,Xinguara - PA,2018,2018,105,Efetivo dos rebanhos,2677,Ovino
6909,6,Município,24,Cabeças,2488,1508407,Xinguara - PA,2019,2019,105,Efetivo dos rebanhos,2677,Ovino
6910,6,Município,24,Cabeças,2169,1508407,Xinguara - PA,2020,2020,105,Efetivo dos rebanhos,2677,Ovino


In [ ]:
table_3939['tipo_producao'] = 'Pecuária'

table_3939_column_names = ['cod_mun', 'mun', 'cod_metrica', 'metrica', 
                           'valor', 'cod_cidade', 'cidade', 'cod_ano', 
                           'ano', 'cod_variavel', 'variavel', 'cod_produto', 
                           'produto', 'tipo_producao']


table_3939.to_csv("/content/drive/MyDrive/Dados/Dados_SIDRA/table_3939-pecuaria.csv", index = False)